In [3]:
import random
import time

from lib.aegis_ashore import AegisAshore
from lib.airbase import Airbase
from lib.aircraft import AircraftPatrolGroup
from lib.launcher import Launcher
from lib.location import Location
from lib.missile import Missile
from lib.port import Port
from lib.radar import Radar
from lib.sam import SAM
from lib.ship import Ship
from lib.simulation import Simulation
from lib.util import launch_missile_salvo, find_entity_by_name

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# This is an example of how to set up a loop to modify different parameter
# values. In this example, we evaluate two different missile types, and
# three different quantities of missiles.

missile_types = ['cruise', 'ballistic']
missile_quantities = [1, 2, 10]

simulation_number = 1
for missile_type in missile_types:
    for missile_quantity in missile_quantities:
        s = Simulation()

        # Toy example: launcher is in Beijing, target is Tokyo.
        l = Launcher(s, 'mylauncher', Location(39.9042, 116.4074), missile_type)
        p = Port(s, 'myport', Location(35.6895, 139.6917))
        
        # Launch missiles in succession, each one second after the other.
        for i in range(missile_quantity):
            s.schedule_event(i, lambda t: l.launch_missile(t, p))

        print()
        print("****Simulation #{}: {} {} missiles".format(
              simulation_number, missile_quantity, missile_type))
        s.print_state()

        while True:
            if not s.process_next_event():
                break

        s.print_state()
        
        simulation_number += 1

TypeError: __init__() missing 1 required positional argument: 'system'

In [ ]:
s = Simulation()

l = Launcher(s, Location(36.9042, 135.4074), 'cruise')
p = Port(s, Location(35.6895, 139.6917))
aegis = FixedDefense(s, Location(0, 0), 'aegis')
sam = FixedDefense(s, Location(0, 0), 'sam')

# This scenario has 20 cruise missiles being launched. Due to the more
# efficient shoot-look-shoot approach used for cruise missiles, 24
# interceptors are approximately sufficient to destroy all of the missiles,
# depending on RNG.
for i in range(20):
    s.schedule_event(0, lambda t: l.launch_missile(t, p))

s.print_state()

while True:
    if not s.process_next_event():
        break

s.print_state()

In [ ]:
# add radar - as we can see, the number of hits 
# decreases, because interceptor accuracy goes up
s = Simulation()

l = Launcher(s, Location(36.9042, 135.4074), 'cruise')
p = Port(s, Location(35.6895, 139.6917))
aegis = FixedDefense(s, Location(0, 0), 'aegis')
sam = FixedDefense(s, Location(0, 0), 'sam')
radar = Radar(s, Location(0, 0))

# 20 cruise missiles
for i in range(20):
    s.schedule_event(0, lambda t: l.launch_missile(t, p))

s.print_state()

while True:
    if not s.process_next_event():
        break

s.print_state()

In [ ]:
# add airbases, aircraft
s = Simulation()

l = Launcher(s, Location(36.9042, 135.4074), 'cruise')
p = Port(s, Location(35.6895, 139.6917))
aegis = FixedDefense(s, Location(0, 0), 'aegis')
sam = FixedDefense(s, Location(0, 0), 'sam')
radar = Radar(s, Location(0, 0))
airbase = Airbase(s, Location(0, 0))
aircraft = AircraftPatrolGroup(s, Location(0, 0))

# 20 cruise missiles
for i in range(20):
    s.schedule_event(0, lambda t: l.launch_missile(t, p))

s.print_state()

while True:
    if not s.process_next_event():
        break

s.print_state()

In [ ]:
# add radar
s = Simulation()

l = Launcher(s, Location(35.9042, 135.4074), 'cruise')
p = Port(s, Location(35.6895, 139.6917))
sam = FixedDefense(s, Location(0, 0), 'sam')
sam2 = FixedDefense(s, Location(0, 0), 'sam')
radar = Radar(s, Location(35.6895, 139.6917))

# First they try to destroy the port, but the defenses are too accurate.
# Then they destroy the radar.
# Then, after the interceptors have reloaded they try again with more success.
for i in range(40):
    s.schedule_event(0, lambda t: l.launch_missile(t, p))
    s.schedule_event(1, lambda t: l.launch_missile(t, radar))
    s.schedule_event(5000, lambda t: l.launch_missile(t, p))

s.print_state()

while True:
    if not s.process_next_event():
        break

s.print_state()

In [ ]:
def fun1():
    s = Simulation()

    # Toy example: launcher is in Beijing, target is Tokyo.
    launchers = [Launcher(s, 'mylauncher', Location(39.9042, 116.4074), 'cruise', 'DF-26') for _ in range(20)]
    p = Port(s, 'myport', Location(35.6895, 139.6917))
    ac = AircraftPatrolGroup(s, 'F-35', Location(35.6895, 139.6917), 'F-35A')
    
    port = find_entity_by_name(s.ports, 'myport')
    
    launch_missile_salvo(s, 0, launchers, {port: 20})
    s.print_state()


    while True:
        if not s.process_next_event():
            break

    s.print_state()
    
fun1()


In [ ]:
# add ships
s = Simulation()

l = Launcher(s, Location(35.9042, 135.4074), 'cruise')
p = Port(s, Location(35.6895, 139.6917))
sam = FixedDefense(s, Location(0, 0), 'sam')
sam2 = FixedDefense(s, Location(0, 0), 'sam')
aegis = FixedDefense(s, Location(0, 0), 'aegis')
radar = Radar(s, Location(35.6895, 139.6917))
airbase = Airbase(s, Location(0, 0))
aircraft = AircraftPatrolGroup(s, Location(0, 0))
ship = Ship(s, Location(0, 0))

# First they try to destroy the port, but the defenses are too accurate.
# Then they destroy the radar.
# Then, after the interceptors have reloaded they try again with more success.
for i in range(80):
    s.schedule_event(0, lambda t: l.launch_missile(t, p))
    s.schedule_event(1, lambda t: l.launch_missile(t, radar))
    s.schedule_event(5000, lambda t: l.launch_missile(t, p))

s.print_state()

while True:
    if not s.process_next_event():
        break

s.print_state()

In [ ]:
s = Simulation()
filename_dict = {
    'airbase': 'data/test/airbases.csv',
    'aircraft': 'data/test/aircraft.csv',
    'launcher': 'data/test/launchers.csv',
    'port': 'data/test/ports.csv',
    'radar': 'data/test/radar.csv',
    'ship': 'data/test/ships.csv',
}
s.load_entities_from_files(filename_dict)

s.print_state()

In [ ]:
def myfun():
    # load real data
    s = Simulation()
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/real1/aircraft.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/real1/ships.csv',
        'launcher': 'data/real1/launchers.csv',
        'radar': 'data/real1/radars.csv',
        'sam': 'data/real1/SAMs.csv',
        'aegis_ashore': 'data/real1/Ashore.csv'
    }
    s.load_entities_from_files(filename_dict)

    # simulate distributed attack: 25% radar, 25% SAMs, 25% ashore, 15% airbases, 10% ports
    # TODO: Ideally the attacks should be planned. I'm doing it in expectation
    # because I'm not sure how to do assign attacks to targets efficiently
    # (brute-forcing is an obvious bad idea).
    
    # Python magic: See https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions.
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    ballistic_target_dict = {
        find_entity_by_name(s.aegis_ashores, 'Akita Aegis Ashore'): 100,
        find_entity_by_name(s.aegis_ashores, 'Yamaguchi Aegis Ashore'): 54,
    }
    
    cruise_target_dict = {
        find_entity_by_name(s.ports, 'Yokohama North Dock'): 10, # This oughta be enough.
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    while True:
        if not s.process_next_event():
            break

    s.print_state()
    
%prun myfun()

In [ ]:
# w/ashore
def limited():
    # load real data
    s = Simulation()
    s.set_render_times([0, 100, 200, 300, 1000, 2000, 3000, 4000, 5000], 'output/limited_ashore')
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/real1/aircraft.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/real1/ships.csv',
        'launcher': 'data/real1/launchers.csv',
        'radar': 'data/real1/radars.csv',
        'sam': 'data/real1/SAMs.csv',
        'aegis_ashore': 'data/real1/Ashore.csv'
    }
    s.load_entities_from_files(filename_dict)

    # simulate escalation-limited radar-focused strike plan
    # no homeland island targets
    # (this is likely an overestimate b/c China wouldn't want
    # to use so much missile inventory on so few targets)
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    # 49 @radar, 30 @SAMs, 45 @airbases, 30 @ports
    ballistic_target_dict = {
        
        # radars - ~10 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 10,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 10,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 10,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 10,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 9,
        
        # airbases - 15 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 15,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 15,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 15,
                
        # SAMs - 15 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 15,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 15,

        # ports - 10 per
        find_entity_by_name(s.ports, 'Naha Port'): 10,
        find_entity_by_name(s.ports, 'Tengan Pier'): 10,
        find_entity_by_name(s.ports, 'White Beach Area'): 10,        
    }
    
    # 138 @radar, 61 @SAMs, 150 @airbases, 123 @ports
    cruise_target_dict = {
        
        # radars - ~28 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 28,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 28,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 28,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 27,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 27,
        
        # airbases - 50 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 50,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 50,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 50,
                
        # SAMs - ~30 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 31,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 30,

        # ports - 41 per
        find_entity_by_name(s.ports, 'Naha Port'): 41,
        find_entity_by_name(s.ports, 'Tengan Pier'): 41,
        find_entity_by_name(s.ports, 'White Beach Area'): 41,   
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    s.print_state()
    
    while True:
        if not s.process_next_event():
            break

    s.print_state()
    s.renderer.render("endstate")
    return s
    
%prun s = limited()

In [ ]:
# limited w/o ashore
def limited():
    # load real data
    s = Simulation()
    s.set_render_times([0, 100, 200, 300, 1000, 2000, 3000, 4000, 5000], 'output/limited')
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/real1/aircraft.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/real1/ships.csv',
        'launcher': 'data/real1/launchers.csv',
        'radar': 'data/real1/radars.csv',
        'sam': 'data/real1/SAMs.csv',
    }
    s.load_entities_from_files(filename_dict)

    # simulate escalation-limited radar-focused strike plan
    # no homeland island targets
    # (this is likely an overestimate b/c China wouldn't want
    # to use so much missile inventory on so few targets)
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    # 49 @radar, 30 @SAMs, 45 @airbases, 30 @ports
    ballistic_target_dict = {
        
        # radars - ~10 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 10,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 10,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 10,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 10,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 9,
        
        # airbases - 15 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 15,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 15,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 15,
                
        # SAMs - 15 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 15,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 15,

        # ports - 10 per
        find_entity_by_name(s.ports, 'Naha Port'): 10,
        find_entity_by_name(s.ports, 'Tengan Pier'): 10,
        find_entity_by_name(s.ports, 'White Beach Area'): 10,        
    }
    
    # 138 @radar, 61 @SAMs, 150 @airbases, 123 @ports
    cruise_target_dict = {
        
        # radars - ~28 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 28,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 28,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 28,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 27,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 27,
        
        # airbases - 50 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 50,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 50,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 50,
                
        # SAMs - ~30 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 31,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 30,

        # ports - 41 per
        find_entity_by_name(s.ports, 'Naha Port'): 41,
        find_entity_by_name(s.ports, 'Tengan Pier'): 41,
        find_entity_by_name(s.ports, 'White Beach Area'): 41,   
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    s.print_state()
    
    while True:
        if not s.process_next_event():
            break

    s.print_state()
    s.renderer.render("endstate")
    return s
    
%prun s = limited()

In [ ]:
# w/ashore, low Chinese missile inventory estimate
def limited():
    # load real data
    s = Simulation()
    s.set_render_times([0, 100, 200, 300, 1000, 2000, 3000, 4000, 5000], 'output/lim_ash_low')
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/real1/aircraft.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/real1/ships.csv',
        'launcher': 'data/altspecs/launchers_low.csv',
        'radar': 'data/real1/radars.csv',
        'sam': 'data/real1/SAMs.csv',
        'aegis_ashore': 'data/real1/Ashore.csv'
    }
    s.load_entities_from_files(filename_dict)

    # simulate escalation-limited radar-focused strike plan
    # no homeland island targets
    # low missile count estimate (77, 254)
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    # 77
    ballistic_target_dict = {
        
        # radars - 5 per (25)
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 5,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 5,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 5,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 5,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 5,
        
        # airbases - 7 per (21)
        find_entity_by_name(s.airbases, 'Naha Air Base'): 7,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 7,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 7,
                
        # SAMs - 7 per (14)
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 7,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 7,

        # ports - ~6 per (17)
        find_entity_by_name(s.ports, 'Naha Port'): 6,
        find_entity_by_name(s.ports, 'Tengan Pier'): 6,
        find_entity_by_name(s.ports, 'White Beach Area'): 5,        
    }
    
    # 253
    cruise_target_dict = {
        
        # radars - 15 per (75)
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 15,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 15,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 15,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 15,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 15,
        
        # airbases - 25 per (75)
        find_entity_by_name(s.airbases, 'Naha Air Base'): 25,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 25,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 25,
                
        # SAMs - 15 per (30)
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 15,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 15,

        # ports - ~24 per (73)
        find_entity_by_name(s.ports, 'Naha Port'): 25,
        find_entity_by_name(s.ports, 'Tengan Pier'): 24,
        find_entity_by_name(s.ports, 'White Beach Area'): 24,   
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    s.print_state()
    
    while True:
        if not s.process_next_event():
            break

    s.print_state()
    s.renderer.render("endstate")
    return s
    
%prun s = limited()

In [ ]:
# w/ashore, high-resilience assets (manually changed in lib; don't do
# this at home, kids) - remember to change it back lol
def limashres():
    # load real data
    s = Simulation()
    s.set_render_times([0, 100, 200, 300, 1000, 2000, 3000, 4000, 5000], 'output/lim_ash_res')
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/real1/aircraft.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/real1/ships.csv',
        'launcher': 'data/real1/launchers.csv',
        'radar': 'data/real1/radars.csv',
        'sam': 'data/real1/SAMs.csv',
        'aegis_ashore': 'data/real1/Ashore.csv'
    }
    s.load_entities_from_files(filename_dict)

    # simulate escalation-limited radar-focused strike plan
    # no homeland island targets
    # (this is likely an overestimate b/c China wouldn't want
    # to use so much missile inventory on so few targets)
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    # 49 @radar, 30 @SAMs, 45 @airbases, 30 @ports
    ballistic_target_dict = {
        
        # radars - ~10 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 10,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 10,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 10,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 10,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 9,
        
        # airbases - 15 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 15,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 15,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 15,
                
        # SAMs - 15 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 15,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 15,

        # ports - 10 per
        find_entity_by_name(s.ports, 'Naha Port'): 10,
        find_entity_by_name(s.ports, 'Tengan Pier'): 10,
        find_entity_by_name(s.ports, 'White Beach Area'): 10,        
    }
    
    # 138 @radar, 61 @SAMs, 150 @airbases, 123 @ports
    cruise_target_dict = {
        
        # radars - ~28 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 28,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 28,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 28,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 27,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 27,
        
        # airbases - 50 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 50,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 50,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 50,
                
        # SAMs - ~30 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 31,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 30,

        # ports - 41 per
        find_entity_by_name(s.ports, 'Naha Port'): 41,
        find_entity_by_name(s.ports, 'Tengan Pier'): 41,
        find_entity_by_name(s.ports, 'White Beach Area'): 41,   
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    s.print_state()
    
    while True:
        if not s.process_next_event():
            break

    s.print_state()
    s.renderer.render("endstate")
    return s
    
%prun s = limashres()

In [ ]:
# w/twice as many aircraft, sm3-equipped ships, in-area radars, in-area sams, aegis ashores
def limdouble():
    # load real data
    s = Simulation()
    s.set_render_times([0, 100, 200, 300, 1000, 2000, 3000, 4000, 5000], 'output/lim_ash_double')
    filename_dict = {
        'airbase': 'data/real1/airbases.csv',
        'aircraft': 'data/altspecs/Aircraft2.csv',
        'port': 'data/real1/ports.csv',
        'ship': 'data/altspecs/Ships2.csv',
        'launcher': 'data/real1/launchers.csv',
        'radar': 'data/altspecs/Radar2.csv',
        'sam': 'data/altspecs/SAMs2.csv',
        'aegis_ashore': 'data/altspecs/Ashore2.csv'
    }
    s.load_entities_from_files(filename_dict)

    # simulate escalation-limited radar-focused strike plan
    # no homeland island targets
    # (this is likely an overestimate b/c China wouldn't want
    # to use so much missile inventory on so few targets)
    ballistic_launchers = [l for l in s.launchers if l.kind == 'ballistic']
    cruise_launchers = [l for l in s.launchers if l.kind == 'cruise']
    print('{} ballistic launchers, {} cruise launchers.'.format(
        len(ballistic_launchers), len(cruise_launchers)))
    
    # <...>
    ballistic_target_dict = {
        
        # radars - ~9 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 9,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 9,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 9,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 9,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 9,
        
        find_entity_by_name(s.radars, 'Yonaguni Radar 2'): 9,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar 2'): 9,
        find_entity_by_name(s.radars, 'Yozadake Radar 2'): 9,
        find_entity_by_name(s.radars, 'Kumejima Radar 2'): 9,
        find_entity_by_name(s.radars, 'Miyakojima Radar 2'): 8,
        
        # airbases - 8 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 8,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 8,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 8,
                
        # SAMs - ~8 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 8,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 8,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1 2'): 7,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2 2'): 7,
        
        # ports - ~4 per
        find_entity_by_name(s.ports, 'Naha Port'): 4,
        find_entity_by_name(s.ports, 'Tengan Pier'): 4,
        find_entity_by_name(s.ports, 'White Beach Area'): 3,        
    }
    
    # 138 @radar, 61 @SAMs, 150 @airbases, 123 @ports
    cruise_target_dict = {
        
        # radars - ~21 per
        find_entity_by_name(s.radars, 'Yonaguni Radar'): 21,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar'): 21,
        find_entity_by_name(s.radars, 'Yozadake Radar'): 21,
        find_entity_by_name(s.radars, 'Kumejima Radar'): 21,
        find_entity_by_name(s.radars, 'Miyakojima Radar'): 20,
        
        find_entity_by_name(s.radars, 'Yonaguni Radar 2'): 22,        
        find_entity_by_name(s.radars, 'Okinoerabujima Radar 2'): 22,
        find_entity_by_name(s.radars, 'Yozadake Radar 2'): 22,
        find_entity_by_name(s.radars, 'Kumejima Radar 2'): 21,
        find_entity_by_name(s.radars, 'Miyakojima Radar 2'): 22,
               
        # airbases - 30 per
        find_entity_by_name(s.airbases, 'Naha Air Base'): 30,
        find_entity_by_name(s.airbases, 'Kadena Air Base'): 30,
        find_entity_by_name(s.airbases, 'Marine Corps Air Station Futenma'): 30,

        # SAMs - ~16 per
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 31,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 30,

        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 1'): 30,
        find_entity_by_name(s.sams, 'Ishigaki SAM Half-Battery 2'): 30,

        # ports - 21 per
        find_entity_by_name(s.ports, 'Naha Port'): 21,
        find_entity_by_name(s.ports, 'Tengan Pier'): 21,
        find_entity_by_name(s.ports, 'White Beach Area'): 21,   
    }
    
    launch_missile_salvo(s, 0, ballistic_launchers, ballistic_target_dict)
    launch_missile_salvo(s, 0, cruise_launchers, cruise_target_dict)

    s.print_state()
    
    while True:
        if not s.process_next_event():
            break

    s.print_state()
    s.renderer.render("endstate")
    return s
    
%prun s = limdouble()